In [12]:
import os

import numpy as np
import tensorflow as tf
from common.extract_model import ExtractModel
from common.freeze_wrapper import write_pb
from feature_detect.src.config import *
from feature_detect.src.feat_data_keras import Data_Gen, Rotate_feed

from tensorboard.plugins.mesh import summary as mesh_summary
from common.plc import build_plc, build_feed_dict

tf.compat.v1.disable_eager_execution()

npz_path = 'F:/ProjectData/mesh_feature/test/test_npz/front'
log_dir = 'F:/ProjectData/mesh_feature/test/log_dir'
batch_size = 1
batch_dim=False
plc, input_names = build_plc(BLOCK_NUM)
model = ExtractModel(CHANNELS, coarse_level=C_LEVEL, fc_dim=4)
output = model(plc)

In [13]:
CKPT_PATH='F:/tf_projects/3D/FeaStNet-master/feature_detect/ckpt'
load_time_dir = '20191031-1754/rutine'  # where to restore the model
ckpt_file = 'ckpt-413'
output = tf.identity(output, 'output_node')
ckpt_full_dir = os.path.join(CKPT_PATH, load_time_dir)
ckpt_full_path = os.path.join(ckpt_full_dir, ckpt_file)
checkpoint = tf.train.Checkpoint(model=model)
ckpt_full_path='F:/tf_projects/3D/FeaStNet-master/feature_detect/ckpt/20191031-1754/rutine/ckpt-413'
status = checkpoint.restore(ckpt_full_path)


model_path = 'F:/tf_projects/3D/FeaStNet-master/feature_detect/freeze_output/1'

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
status.initialize_or_restore(sess)
# for node in tf.train.list_variables(tf.train.latest_checkpoint(ckpt_full_dir)):
#     print(node)

W1101 17:36:28.903074 16264 util.py:144] Unresolved object in checkpoint: (root).optimizer
W1101 17:36:28.903074 16264 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W1101 17:36:28.904072 16264 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W1101 17:36:28.904072 16264 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W1101 17:36:28.905069 16264 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W1101 17:36:28.905069 16264 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W1101 17:36:28.905069 16264 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.FC_output.kernel
W1101 17:36:28.906066 16264 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.FC_output.bias
W1101 17:36:28.906066 16264 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).model.Blocks.0.conv1.W
W1101 

In [14]:
if os.path.exists(model_path):
    import shutil
    
    shutil.rmtree(model_path)
else:
    os.mkdir(model_path)


def build_input_info(plc_dict):
    tensor_infos = {'vertice': plc['vertice']}
    adj_infos = {'adj_%d' % i: adj_plc for i, adj_plc in enumerate(plc['adjs'])}
    perm_infos = {'perm_%d' % i: perm_plc for i, perm_plc in enumerate(plc['perms'])}
    tensor_infos.update(adj_infos)
    tensor_infos.update(perm_infos)
    return tensor_infos


# ordinary model
tf.compat.v1.saved_model.simple_save(sess, model_path, build_input_info(plc), {'output_node': output})
write_pb(input_saved_model_dir=model_path,
         output_graph_filename="../output_graph.pb")

ValueError: At least two variables have the same name: extract_model/block/conv__mesh/W

In [ ]:
data_gen = Data_Gen('F:/ProjectData/mesh_feature/Case_npz/back')
rf = Rotate_feed(
    rot_num=2,
    rot_range=[np.pi / 18., np.pi / 18., np.pi / 18.],
    angle_fixed=True,
    data_gen=data_gen
)
epoch_end = False

# while (not epoch_end):
feed_numpy, epoch_end = rf.rotate_case()
x = feed_numpy['vertice'] #[b,n,3]
adjs = feed_numpy['adjs'] #[[n,K]]
perms = feed_numpy['perms'] #[[n]]
mask = feed_numpy['mask'] #[[n]]

feed_dict = build_feed_dict(plc, x, adjs, perms)
result = sess.run(output, feed_dict=feed_dict)
output = np.reshape(result, [-1, FEAT_CAP, 3])
result = output[:,mask,:]  #[1,f_num,3]
print(result)
sess.close()
##########

In [ ]:
# Camera and scene configuration.
config_dict = {
    'material': {
        'cls': 'PointsMaterial',
        'size': 1,
        # 'color':0x000fff
    }
}

points_v = x
points_f = result
points = np.concatenate([points_v, points_f], axis=1)

colors_v = np.ones_like(points_v) * [0, 0, 255]
colors_f = np.ones_like(points_f) * [0, 255, 0]
colors = np.concatenate([colors_v, colors_f], axis=1)
    
tf.compat.v1.disable_eager_execution()

points_tensor = tf.compat.v1.placeholder(tf.float32, points.shape)
colors_tensor = tf.compat.v1.placeholder(tf.int32, colors.shape)

summary = mesh_summary.op(
    'v_color_tensor',
    vertices=points_tensor,
    colors=colors_tensor,
    config_dict=config_dict
)

# Create summary writer and session.
writer = tf.compat.v1.summary.FileWriter(log_dir)
sess = tf.compat.v1.Session()


summaries = sess.run([summary], feed_dict={
    points_tensor: points,
    colors_tensor: colors,
})
# Save summaries.
for summary in summaries:
    writer.add_summary(summary)
    
